**Commands to start Development Environment**
1. Install Docker-Desktop, Login
2. docker run (Jupyter notebook - should update command once loggeing out)
    1. docker run --name devenv -it --entrypoint=/bin/bash -p 8880:8888 -v C:\users\srmenon\Documents:/notebooks jupyter/tensorflow-notebook
    
3. cd /notebooks/
4. nohup jupyter notebook --port 8880 --allow-root &
5. jupyter notebook list

In [ ]:
!pip install twitter
!pip install feedparser

In [ ]:
# Authenticate with Twitter & search
# https://pypi.org/project/twitter/
# https://developer.twitter.com/en/apps/17019807
from twitter import *
t = Twitter(auth=OAuth(consumer_key="nDwKHutvaskyGecZkCS1SiYY8", consumer_secret="zlQdJIuFUGGglbMO2kYMk6oNLFztzipxhRANNz5q5gqfn1tGb1", token="73568030-GqfoqkfWRhqhARS9uzz51vg1UGSV7AGRaOxDkAurZ", token_secret="1kiHGfyDfO7gcaJbGFnr3lnLA3vXc8ti9hJhj1lnTTvEm"))
t.search.tweets(q="$MSFT")

In [ ]:
# Download latest info about ticker from Y! Finance Feed 
# https://query1.finance.yahoo.com/v7/finance/chart/MSFT?&interval=5m
import json
from urllib.request import urlopen
json.load(urlopen("https://query1.finance.yahoo.com/v7/finance/chart/MSFT?&interval=5m"))

In [ ]:
# Use feedparser to download and parse reuters feed for business news
# https://www.pythonforbeginners.com/feedparser/using-feedparser-in-python
# http://feeds.reuters.com/reuters/businessNews
import feedparser
d = feedparser.parse("http://feeds.reuters.com/reuters/businessNews")
print([l.title for l in d["entries"]])

**Commands to start Kafka Cluster**

https://success.docker.com/article/getting-started-with-kafka

Use linux container mode (else bridging will fail for overlay network)

1. docker swarm init
2. docker network create -d overlay --attachable kafka-net
3. docker service create --network kafka-net --name=zookeeper  --publish 2181:2181 qnib/plain-zookeeper:latest
4. docker service create --network kafka-net --name=zkui --publish 9090:9090 qnib/plain-zkui:latest
5. Visit http://localhost:9090/  # Login using admin/manager
6. docker service create --network kafka-net --name broker --publish 9092:9092 --hostname="{{.Service.Name}}.{{.Task.Slot}}.{{.Task.ID}}" -e KAFKA_BROKER_ID={{.Task.Slot}} -e ZK_SERVERS=tasks.zookeeper qnib/plain-kafka:latest
7. docker ps -q --filter "label=com.docker.swarm.service.name=broker"
8. docker exec -t -e JMX_PORT="" f6dc03b145c0 /opt/kafka/bin/kafka-topics.sh --zookeeper tasks.zookeeper:2181 --partitions=1 --replication-factor=1 --create --topic test
9. docker service create --network kafka-net --name manager -e ZOOKEEPER_HOSTS=tasks.zookeeper --publish=9000:9000 qnib/plain-kafka-manager:latest

http://localhost:9000/clusters/kafka

OR EQUIVALENTLY

https://docs.docker.com/compose/gettingstarted/

1. docker-compose up -d KafkaSetup.yml 

In [ ]:
!pip install kafka-python

In [ ]:
from kafka import KafkaConsumer
from json import loads

consumer = KafkaConsumer(
    'test',
     bootstrap_servers=['broker.1.5ftnqdo6u8ee4888cazael8df:9092'],
     api_version= (0,11),
     auto_offset_reset='earliest',
     enable_auto_commit=True,
     #group_id='my-group',
     value_deserializer=lambda x: loads(x.decode('utf-8')))

for message in consumer:
    message = message.value
    print('{0} added'.format(message))

In [ ]:
from time import sleep
from json import dumps
from kafka import KafkaProducer

producer = KafkaProducer(
    bootstrap_servers=['broker.1.5ftnqdo6u8ee4888cazael8df:9092'],
    api_version= (0,11),
    value_serializer=lambda x: dumps(x).encode('utf-8'))

for e in range(4):
    print("Processing "+str(e)+"\n")
    data = {'number' : e}
    producer.send('test', value=data)
    sleep(1)
    
producer.flush()

**Troubleshooting the network**

https://success.docker.com/article/troubleshooting-container-networking

1. docker run -it --network container:devenv nicolaka/netshoot
2. lookup broker.1.c3xailkxvbttt0cdph1n303i1 [Successful]
3. nc -zvw2 @backend_ip <listening_port>
4. nslookup <IP_address>

In [ ]:
# Troubleshooting the network

import socket
#print(socket.gethostbyname('www.google.com'))
#print(socket.gethostbyname('0410cdf71fcc')) #Succeeds now (add notebook container to the kafka-net network)
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
try:
    sock.connect(('10.0.1.73',9092))
except Exception as e:
    print("something's wrong with %s:%d. Exception is %s" % (address, port, e))
finally:
    sock.close()

**PySPARK Begins Here**

1. docker run -d -p 8888:8888 -v C:\Users\srmenon\Documents:/home/jovyan/work --name spark jupyter/pyspark-notebook
2. docker container ls
3. docker exec 8602565b15e7 jupyter notebook list
4. docker network connect kafka-net spark


In [ ]:
# Testing spark

import pyspark
sc = pyspark.SparkContext('local[*]')

# do something to prove it works
rdd = sc.parallelize(range(1000))
rdd.takeSample(False, 5)

Create a consumer group to use the spark context

1. docker exec -t -e JMX_PORT="" 8cbb69e65d13 /opt/kafka/bin/kafka-console-consumer.sh --bootstrap-server broker.1.5ftnqdo6u8ee4888cazael8df:9092 --topic test --consumer-property groupd.id=test-consumer-group
2. start the producer above (So that data is pupmed in post the group creation)

In [ ]:
# Connect pyspark to kafka and start stream processing
# https://spark.apache.org/docs/latest/streaming-programming-guide.html

import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
print(conf.toDebugString())
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60)
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper.1.252d8i3iyqh7keyobvpjj4dz8:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'test':1})
lines = kafkaStream.map(lambda x: x[1])
lines.pprint()
ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()

**Tensorflow Model Serving with Docker**

https://www.tensorflow.org/tfx/serving/docker

Running an instance of TSX Server
1. docker pull tensorflow/serving
2. git clone https://github.com/tensorflow/serving
3. docker run -t --rm -p 8501:8501 -v "C:\Users\srmenon\Documents\Jarvis\serving\tensorflow_serving\servables\tensorflow\testdata\saved_model_half_plus_two_cpu:/models/half_plus_two" -e MODEL_NAME=half_plus_two  tensorflow/serving &
4. curl -d "{\"instances\": [1.0, 2.0, 5.0]}" -g -X POST http://localhost:8501/v1/models/half_plus_two:predict
5. docker network connect kafka-net xenodochial_tesla

In [ ]:
# PySpark Code to call TSX
import requests
r = requests.post(
    "http://10.0.1.48:8501/v1/models/half_plus_two:predict",
    "{\"instances\": [1.0, 2.0, 5.0]}")
# Am able to reach the server using ip address - but name resolution is failing
print(r.text)

In [ ]:
# PySpark streaming from Kafka with calls to TSX
# DStream transformations : http://sungsoo.github.io/2015/04/06/transformations-on-dstreams.html

import os
import requests
import json

os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
print(conf.toDebugString())
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper.1.252d8i3iyqh7keyobvpjj4dz8:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'test':1})
lines = kafkaStream.map(lambda x: x[1])
lines.pprint()
lines = lines.map(lambda a: str(json.loads(a)["number"]))
lines.pprint()
lines = lines.reduce(lambda v,agg: agg+", "+v)
lines.pprint()
lines = lines.map(lambda a: requests.post(
        "http://10.0.1.48:8501/v1/models/half_plus_two:predict",
        "{\"instances\": ["+a+"]}").text)
lines.pprint()
ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()


**Setup Redis Image and DB**

https://koukia.ca/installing-redis-on-windows-using-docker-containers-7737d2ebc25e
https://github.com/MicrosoftArchive/redis/releases

1. docker pull redis
2. docker run --name redis-store -d redis
3. docker network connect kafka-net redis-store
4. goto C:\Program Files\Redis
5. redis-cli -h localhost/127.0.0.1 -p 6379
6. set key value
7. get key

In [ ]:
# Write to Redis from Python
# !pip install redis
import redis
r = redis.StrictRedis(host = 'redis-store', port = 6379)
#r.set("Radhika",100)
print(r.get("171.5"))

In [ ]:
# Write to Redis from within PySpark context

import os
import requests
import json
import redis

os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[2] pyspark-shell'

import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

def SetVal(x):
    r = redis.StrictRedis(host = 'redis-store', port = 6379)
    r.set(x,int(x))

conf = SparkConf().set("spark.jars","/home/jovyan/work/spark-streaming-kafka-0-8-assembly_2.11-2.4.4.jar")
print(conf.toDebugString())
sc = SparkContext(conf=conf)
sc.setLogLevel("WARN")
ssc = StreamingContext(sc,60) # 60 is the batch interval :)
kafkaStream = KafkaUtils.createStream(
    ssc=ssc, 
    zkQuorum='zookeeper.1.252d8i3iyqh7keyobvpjj4dz8:2181', 
    groupId='test-consumer-group',#, #Consumer Group 
    topics={'test':1})
lines = kafkaStream.map(lambda x: x[1])
lines.pprint()
lines = lines.map(lambda a: str(json.loads(a)["number"]))
lines.pprint()
lines = lines.reduce(lambda v,agg: agg+", "+v)
lines.pprint()
lines = lines.map(lambda a: requests.post(
        "http://10.0.1.48:8501/v1/models/half_plus_two:predict",
        "{\"instances\": ["+a+"]}").text)
lines.pprint()
lines = lines.flatMap(lambda x: json.loads(x)["predictions"])
lines.pprint()
lines = lines.map(lambda x: SetVal(x))
lines.pprint()

ssc.start()  
print("Going to wait termination")
ssc.awaitTermination()